In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from tifffile import memmap

from caImageAnalysis import BrukerFish
from caImageAnalysis.caiman_process import caiman_mcorr
from caImageAnalysis.mesm import *
from caImageAnalysis.visualize import visualize_images, visualize_volumes

pd.options.display.max_colwidth = 120

### Change the exp_folder to the path of the experiment

In [2]:
# Change this path to the experiment folder
exp_folder = r'/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809'

In [3]:
# Initialize the fish. 
# If custom 2P, use VolumeFish. If Bruker 2P, use BrukerFish.

# fish = VolumeFish(exp_folder)
fish = BrukerFish(exp_folder)
fish.data_paths

{'raw': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/vagal_L-001'),
 'frametimes': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/frametimes.txt'),
 'raw_image': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/._ch2.tif'),
 'rotated': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/raw_rotated.tif'),
 'opts': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/opts.pkl'),
 'mesmerize': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/mesmerize-batch'),
 'log': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/vagal_L-001/vagal_L-001.xml'),
 'voltage_output': PosixPath('/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/vagal_L-001/vagal_L-001_Cycle00001_VoltageOutp

## Pick a plane to apply movement correction

In [4]:
iw = visualize_volumes(fish)
iw.show()

RFBOutputContext()

/Users/minelarinel/mambaforge/envs/mescore/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [4]:
plane = 1

## Set parameters

In [5]:
# motion correction parameters
strides = (80, 80)          # start a new patch for pw-rigid motion correction every x pixels
overlaps = (30, 30)         # overlap between pathes (size of patch strides+overlaps)
max_shifts = (20, 20)          # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = 20     # maximum shifts deviation allowed for patch with respect to rigid shifts
pw_rigid = True             # flag for performing non-rigid motion correction

In [6]:
opts_dict = {'strides': strides,
            'overlaps': overlaps,
            'max_shifts': max_shifts,
            'max_deviation_rigid': max_deviation_rigid,
            'pw_rigid': pw_rigid}

## Motion correct

In [7]:
mcorr_img = caiman_mcorr(fish, plane=plane, **opts_dict)

  0%|          | 0/1 [00:00<?, ?it/s]

Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/img_stack_1/image_els__d1_512_d2_512_d3_1_order_F_frames_237.mmap


100%|██████████| 1/1 [00:00<00:00,  7.22it/s]


Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/img_stack_1/image_els__d1_512_d2_512_d3_1_order_F_frames_237.mmap
/Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/img_stack_1/memmap_d1_512_d2_512_d3_1_order_C_frames_237.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/img_stack_1/memmap_d1_512_d2_512_d3_1_order_C_frames_237.mmap


In [8]:
# Compare the results
orig = memmap(fish.data_paths['volumes'][str(plane)]['image'])
iw = visualize_images([orig, mcorr_img], names=['original', 'mcorr'])
iw.show()

RFBOutputContext()

/Users/minelarinel/mambaforge/envs/mescore/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


## Optimize parameters with Mesmerize

In [ ]:
# Pick a plane to optimize motion correction
plane = 1

### Motion correction parameters

In [9]:
# Set combination of values to try
strides = [80, 60]  # start a new patch for pw-rigid motion correction every x pixels
overlaps = [30, 15]  # overlap between patches (size of patch strides+overlaps)
max_shifts = [20, 30]  # maximum allowed rigid shifts (in pixels)
max_deviation_rigid = [20]  # maximum shifts deviation allowed for patch with respect to rigid shifts

In [10]:
img_path = fish.data_paths['volumes'][str(plane)]['image']
mes_df = add_mcorr(fish, [img_path], grid=True, 
                         strides=strides, 
                         overlaps=overlaps,
                         max_shifts=max_shifts,
                         max_deviation_rigid=max_deviation_rigid)

Batch does not exist. Creating batch.pickle


In [11]:
mes_df

,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,21a33383-efb4-4772-8c60-71d1ce32e4e4
1,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (30, 30), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,65ce280d-78f2-425f-bdaa-8f473f589747
2,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (15, 15), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,27d6206b-bafa-48a9-aaf8-d2f874b02f6d
3,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (80, 80), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,f4b4b52a-b592-4b31-8d76-5f8db74181de
4,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (60, 60), 'overlaps': (30, 30), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,af3a45a8-fd37-46a1-9c70-5ad213d8a717
5,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (60, 60), 'overlaps': (30, 30), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,149f822d-d228-4a29-9e77-abb67a79f008
6,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (60, 60), 'overlaps': (15, 15), 'max_shifts': (20, 20), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,6a9a6fee-c58f-4979-93b4-66de64d23a95
7,mcorr,img_stack_1,img_stack_1/image.tif,"{'main': {'strides': (60, 60), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 20, 'pw_rigid': ...",None,2023-08-13T19:29:30,None,None,None,dbcd497f-0877-4c88-af04-2dc20c6f17a8


In [12]:
# See the unique parameters between rows with the same item_name
diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])
diffs

/var/folders/48/8y2gqcvx5q5575kwkzngl6qw0000gn/T/ipykernel_45080/688498701.py:2: FutureWarning: You are trying to use the following experimental feature, this may change in the future without warning:
CaimanDataFrameExtensions.get_params_diffs
This feature is new and the might improve in the future

  diffs = mes_df.caiman.get_params_diffs(algo="mcorr", item_name=mes_df.iloc[0]["item_name"])


0    {'overlaps': (30, 30), 'strides': (80, 80), 'max_shifts': (20, 20)}
1    {'overlaps': (30, 30), 'strides': (80, 80), 'max_shifts': (30, 30)}
2    {'overlaps': (15, 15), 'strides': (80, 80), 'max_shifts': (20, 20)}
3    {'overlaps': (15, 15), 'strides': (80, 80), 'max_shifts': (30, 30)}
4    {'overlaps': (30, 30), 'strides': (60, 60), 'max_shifts': (20, 20)}
5    {'overlaps': (30, 30), 'strides': (60, 60), 'max_shifts': (30, 30)}
6    {'overlaps': (15, 15), 'strides': (60, 60), 'max_shifts': (20, 20)}
7    {'overlaps': (15, 15), 'strides': (60, 60), 'max_shifts': (30, 30)}
Name: params, dtype: object

### Run motion correction with the given parameter space

In [13]:
mes_df = load_mesmerize(fish)
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

Batch exists. Loading batch.pickle
Running 21a33383-efb4-4772-8c60-71d1ce32e4e4 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/mesmerize-batch/21a33383-efb4-4772-8c60-71d1ce32e4e4/21a33383-efb4-4772-8c60-71d1ce32e4e4-image_els__d1_512_d2_512_d3_1_order_F_frames_237.mmap
finished computing correlation image
Running 65ce280d-78f2-425f-bdaa-8f473f589747 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/mesmerize-batch/65ce280d-78f2-425f-bdaa-8f473f589747/65ce280d-78f2-425f-bdaa-8f473f589747-image_els__d1_512_d2_512_d3_1_order_F_frames_237.mmap
finished computing correlation image
Running 27d6206b-bafa-48a9-aaf8-d2f874b02f6d with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename /Volumes

In [14]:
# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

True
True
True
True
True
True
True
True


### Visualize outputs

In [4]:
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/mesmerize-batch/21a33383-efb4-4772-8c60-71d1ce32e4e4/21a33383-efb4-4772-8c60-71d1ce32e4e4-image_els__d1_512_d2_512_d3_1_order_F_frames_237.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/mesmerize-batch/65ce280d-78f2-425f-bdaa-8f473f589747/65ce280d-78f2-425f-bdaa-8f473f589747-image_els__d1_512_d2_512_d3_1_order_F_frames_237.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/mesmerize-batch/27d6206b-bafa-48a9-aaf8-d2f874b02f6d/27d6206b-bafa-48a9-aaf8-d2f874b02f6d-image_els__d1_512_d2_512_d3_1_order_F_frames_237.mmap
Decode mmap filename /Volumes/Rapid/bruker2P_data/elavl3H2BGCaMP8m_8dpf_fed_glucose_500mM_1_20230809/mesmerize-batch/f4b4b52a-b592-4b31-8d76-5f8db74181de/f4b4b52a-b592-4b31-8d76-5f8db74181de-image_els__d1_512_d2_512_d3_1_order_F_f

RFBOutputContext()

/Users/minelarinel/mambaforge/envs/mescore/lib/python3.10/site-packages/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


In [ ]:
mcorr_iw.plot.canvas.close()

In [19]:
# Visualize the difference from the mean
mcorr_iw = visualize_diff(mes_df)
mcorr_iw.show()

Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\c9f646b6-e1e0-4d2c-bae1-19cca7c77e23\c9f646b6-e1e0-4d2c-bae1-19cca7c77e23-image_els__d1_166_d2_250_d3_1_order_F_frames_833.mmap
Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\d6f64a99-45e4-4b60-be73-37d7375de6ac\d6f64a99-45e4-4b60-be73-37d7375de6ac-image_els__d1_166_d2_250_d3_1_order_F_frames_833.mmap
Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\e92c5fbe-6669-466a-900e-d9b83e1cf8f6\e92c5fbe-6669-466a-900e-d9b83e1cf8f6-image_els__d1_166_d2_250_d3_1_order_F_frames_833.mmap
Decode mmap filename C:\Users\minel\Documents\data\elavl3H2BGCaMP8m_7dpf_fed_glucose_500mM_25_20221227\postgavage_vagal_L\mesmerize-batch\7809fe1b-0f60-4cb9-ab8c-cc3084c0e9a3\7809fe1b-0f60-4cb9-ab8c-cc3

RFBOutputContext()

In [20]:
mcorr_iw.plot.canvas.close()

### Choose an index with the best parameters

In [13]:
indices = []

In [14]:
df = load_mesmerize(fish)
mes_df = clean_mesmerize(df, indices)
mes_df

Batch exists. Loading batch.pickle


,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid


## Run motion correction on all planes

In [6]:
fish.process_mesmerize_filestructure()
mes_df = add_volume(fish, algo='mcorr')
mes_df

Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle
Batch exists. Loading batch.pickle


,algo,item_name,input_movie_path,params,outputs,added_time,ran_time,algo_duration,comments,uuid
0,mcorr,img_stack_3,img_stack_3\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",{'mean-projection-path': 81e9b407-0f06-419d-bde6-a34491054277\81e9b407-0f06-419d-bde6-a34491054277_mean_projection.n...,2023-05-19T13:25:30,2023-05-19T13:27:10,47.31 sec,None,81e9b407-0f06-419d-bde6-a34491054277
1,mcorr,img_stack_0,img_stack_0\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,4368db23-af24-4635-b1e5-5ed9917c3b12
2,mcorr,img_stack_1,img_stack_1\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,709f79b0-3e1c-4c70-b33e-67aaa2161c83
3,mcorr,img_stack_2,img_stack_2\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,d75ab8d1-c458-4f99-bf43-6d5aee0e2e92
4,mcorr,img_stack_4,img_stack_4\image.tif,"{'main': {'strides': (40, 40), 'overlaps': (15, 15), 'max_shifts': (30, 30), 'max_deviation_rigid': 30, 'pw_rigid': ...",None,2023-05-19T13:35:13,None,None,None,b9273412-6809-414a-8b3e-669f9533dc9c


In [7]:
run_mesmerize(mes_df)
mes_df = mes_df.caiman.reload_from_disk()

# See the rows that were successfully run
for output in mes_df.loc[:, 'outputs']:
    print(output['success'])

Running 4368db23-af24-4635-b1e5-5ed9917c3b12 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\4368db23-af24-4635-b1e5-5ed9917c3b12\4368db23-af24-4635-b1e5-5ed9917c3b12-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
finished computing correlation image
Running 709f79b0-3e1c-4c70-b33e-67aaa2161c83 with local backend
starting mc
mc finished successfully!
computing projections
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\709f79b0-3e1c-4c70-b33e-67aaa2161c83\709f79b0-3e1c-4c70-b33e-67aaa2161c83-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
finished computing correlation image
Running d75ab8d1-c458-4f99-bf43-6d5aee0e2e92 with local backend
starting mc
mc finished successfully!
computing projecti

### Visualize outputs

In [8]:
mes_df = load_mesmerize(fish)
mcorr_iw = visualize_mesmerize(mes_df, 'mcorr')
mcorr_iw.show()

Batch exists. Loading batch.pickle
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\81e9b407-0f06-419d-bde6-a34491054277\81e9b407-0f06-419d-bde6-a34491054277-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\4368db23-af24-4635-b1e5-5ed9917c3b12\4368db23-af24-4635-b1e5-5ed9917c3b12-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_vagal_R\mesmerize-batch\709f79b0-3e1c-4c70-b33e-67aaa2161c83\709f79b0-3e1c-4c70-b33e-67aaa2161c83-image_els__d1_400_d2_400_d3_1_order_F_frames_480.mmap
Decode mmap filename C:\Users\minel\Documents\data\vglut2dsRed_elavl3H2BGCaMP8m_6dpf_unfed_glucose_500mM_9_20221024\postgavage_v

RFBOutputContext()

In [9]:
mcorr_iw.plot.canvas.close()